# Profiling

Understanding and locating the bottlenecks of a program is crucial for its optimization, since these bottlenecks (or hot spots) are the parts where most of the running is spent. The profile of a program is essentially composed of a number of counters, timers and locations that together describe how much of its running time the program spends at different regions.
In general, there are two approaches to profiling: deterministic profiling and statistical profiling. They differ not only in the precision of their results but also in the overhead they incur on the runtime and how their insights into the resource consumption of a program are gained.

__Deterministic profiling__ makes use of trace functions that are inserted into the program that meant to be profiled. They enable the profiler to measure the elapsed time between certain events, such as function calls, function returns, thrown exceptions, and more. 

__Statistical profililing__ only provides approximate information because these profiles sample the stack frames at regular time intervals, which allows this kind of profiler to analyse them statistically and thereby extract meaningful metrics for the profiled program. Even though the overhead brought about by profiling is usually not insignificant, statistical profilers involve much less overhead than deterministic ones, which comes at the price of less precise profiles.

Conveniently, the Python interpreter already includes the mechanisms that are necessary for deterministic profiling. Callback functions, i.e. a function passed as an argument to other functions that are expected to call this passed function, can be inserted at program or interpreter level for various events. Additionally, due the fact that the Python interpreter already causes significant overhead, these callbacks add little overhead on top of that compared to compiled languages.

The __call count__, which counts how often a certain function is called, __timers__ and __memory usage__ are common metrics in profilers that can shed light on various program characteristics:
* __Hot loops__, i.e. loops with a high number of iterations, can be identified and subsequently optimized
* Functions that potentially benefit from __in-lining__ can be spotted by their high call count
* Untypical call counts can indicate __bugs__ 
* High memory usage can help to identify __memory leaks__

## `timeit`

The `timeit` magic command in the Jupyter allows to measure the time execution of small programs or even single instructions by using the module `timeit`. It is not necessary to import anything explicitely and it is both a cell and a line magic command. `timeit` temporarily turns off garbage collection during the timing, which leads to different results than those obtained from `time`, but GC can be re-enabled as the first statement in the setup string. Practically, `timeit` automatically adjusts the number of repetitions, if it is not stated explicitely. It must be noted that, strictly speaking, `timeit` is to be used for __benchmarking__ purposes only, i.e. the program in execution is neither inspected nor instrumented.

___References___
* [timeit — Measure execution time of small code snippets](https://docs.python.org/3/library/timeit.html)

In [1]:
%timeit?

Docstring:
Time execution of a Python statement or expression

Usage, in line mode:
  %timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement
or in cell mode:
  %%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code
  code
  code...

Time execution of a Python statement or expression using the timeit
module.  This function can be used both as a line and cell magic:

- In line mode you can time a single-line statement (though multiple
  ones can be chained with using semicolons).

- In cell mode, the statement in the first line is used as setup code
  (executed but not timed) and the body of the cell is timed.  The cell
  body has access to any variables created in the setup code.

Options:
-n<N>: execute the given statement <N> times in a loop. If <N> is not
provided, <N> is determined so as to get sufficient accuracy.

-r<R>: number of repeats <R>, each consisting of <N> loops, and take the
best result.
Default: 7

-t: use time.time to measure the time, which is the default on Unix.
This f

In [2]:
s0 = 'This is a string'
s1 = 'and this is another string'
s2 = 'with the number' 
i = 1337

In [3]:
%timeit -n 1_000_000 -r 10 s = f'{s0}, {s1} {s2} {i}.'

212 ns ± 0.976 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)


In [4]:
%timeit -n 1_000_000 -r 10 s = s0 + ', ' + s1 + ' ' + s2 + ' ' + str(i) + '.'

405 ns ± 0.888 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)


In [5]:
%timeit -n 1_000_000 -r 10 s = '{}, {} {} {}.'.format(s0, s1, s2, str(i))

441 ns ± 1.21 ns per loop (mean ± std. dev. of 10 runs, 1,000,000 loops each)


In [6]:
!python3 -m timeit -s "s0 = 'This is a string'; s1 = 'and this is another string'; s2 = 'with the number'; i = 1337" "s = f'{s0}, {s1} {s2} {i}.'"

20000000 loops, best of 5: 11.2 nsec per loop


The reason for stating the best time time is that the lowest value gives a lower bound for how fast the given code snippet can be executed on the current computer, while higher values are typically not caused by variability in the speed of the Python interpreter, but by other processes interfering with the timers' accuracy.

## `cProfile` & `profile`

`cProfile` and `profile` are both deterministic profilers of Python programs and part of the standard library. The statistics that they produce with their profiles can be formatted into reports with  `pstats` module. To be precise, these two profilers are actually different implementations of the same profiling interface. As the name suggests, `cProfile` is a C extension with reasonable overhead and therefore it is a viable choice also for programs with a longer runtime. On the other hand, `profile` is a pure Python module, which implies significant overhead compared to a C extension, whose interface inspired that of `cProfile`.


___References___
* [The Python Profilers](https://docs.python.org/3/library/profile.html)

$$\sum_{k=0}^{\infty} \frac{(-1)^k}{2k+1} = 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} - \dotsb = \frac{\pi}{4}$$

In [7]:
#%%writefile profiling/sleibniz.py

#import cProfile


def numerator(i):
    return (-1)**i


def denominator(i):
    return 1.0/(2*i + 1)

    
def sleibniz(n=1_000_000):
    s = 0.0
    for i in range(n):
        s += numerator(i)*denominator(i)
    return 4*s


#cProfile.run('sleibniz()')

In [8]:
!python3 profiling/sleibniz.py

python3: can't open file '/home/fs70824/trainee19/python4hpc/profiling/sleibniz.py': [Errno 2] No such file or directory


In [9]:
%prun sleibniz()

         2000004 function calls in 0.918 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.379    0.379    0.918    0.918 2544042676.py:14(sleibniz)
  1000000    0.371    0.000    0.371    0.000 2544042676.py:6(numerator)
  1000000    0.167    0.000    0.167    0.000 2544042676.py:10(denominator)
        1    0.000    0.000    0.918    0.918 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.918    0.918 <string>:1(<module>)

In [10]:
def fleibniz(n=1_000_000):
    s = 0.0
    for i in range(n):
        s += ((-1)**i)/(2*i + 1)
    return 4*s


def ffleibniz(n=1_000_000):
    s = 0.0
    for i in range(n):
        s += ((-1)*(i%2))/(2*i + 1)
    return 4*s

In [11]:
%prun fleibniz()

         4 function calls in 0.407 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.407    0.407    0.407    0.407 2914762232.py:1(fleibniz)
        1    0.000    0.000    0.407    0.407 {built-in method builtins.exec}
        1    0.000    0.000    0.407    0.407 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [12]:
%timeit fleibniz(100_000)

37.6 ms ± 41 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit ffleibniz(100_000)

13.3 ms ± 6.06 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit sleibniz(100_000)

53.2 ms ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## `pstats`

The pstats module’s Stats class has a variety of methods for manipulating and printing the data saved into a profile results file:

___References___
* [Profiler Data Analysis with pstats](https://docs.python.org/3/library/profile.html#module-pstats)

In [15]:
import cProfile
import pstats
from performance.sleibniz import sleibniz


# Create 5 set of stats
filenames = []
for i in range(1):
    filename = f'cProfile_stats_{i}.stats'
    cProfile.run('sleibniz(100_000)', filename)
    
#cProfile.run('print %d, sleibniz(10_000)' % i, filename)

# Read all 5 stats files into a single object
stats = pstats.Stats('cProfile_stats_0.stats')
#for i in range(1, 2):
#    stats.add(f'cProfile_stats_{i}.stats')

# Clean up filenames for the report
stats.strip_dirs()

# Sort the statistics by the cumulative time spent in the function
stats.sort_stats('cumulative')

stats.print_stats()

ModuleNotFoundError: No module named 'performance'

## `line_profiler`

While the information obtained by `cProfile` already gives a good overview of how much time is spent in a function and where the hotspots are located, which is in many cases sufficient to optimize the code to a reasonable degree, however, sometimes the reason for poor performance lies in a single line of code. In scientific applications, with their loop-heavy code and frequent as well as expensive calls to libraries, this is often the case. Such a single line hotspot is never explicitely shown by `cProfile` since it might not even contain a function call. 
This is where the `line_profiler` can play its strengths. It takes specific functions to profile, and it will time the execution of each individual line inside those functions. Typically, a line profiler is useful to inspect the runtimes of lines in a few functions because digging through the timings of each line of code would simply be overwhelming.


___References___
* [PyPi line-profiler](https://pypi.org/project/line-profiler/)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun?

In [ ]:
%%writefile profiling/matmul.py

import numpy as np


n = 29
A = np.random.rand(n, n)
B = np.random.rand(n, n)
C = np.zeros_like(A)


def matmul_ijk(A, B, C):
    n = np.shape(A)[0]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
                
                
def matmul_ikj(A, B, C):
    n = np.shape(A)[0]
    for i in range(n):
        for k in range(n):
            x = A[i][k]
            for j in range(n):
                C[i][j] += x * B[k][j]

In [ ]:
import numpy as np


n = 100
A = np.random.rand(n, n)
B = np.random.rand(n, n)
C = np.zeros_like(A)

In [ ]:
%timeit -r 30 -n 10 matmul_ijk(A, B, C)

In [ ]:
%timeit -r 30 -n 10 matmul_ikj(A, B, C)

In [ ]:
%lprun -f matmul_ijk matmul_ijk(A, B, C)

In [ ]:
%lprun -f matmul_ikj_rawster matmul_ikj_rawster(A, B, C)

In [ ]:
B = np.random.rand(10, 5)
print(B.shape)
print(len(B[:,0]))

## `memory_profiler`

One more important aspect of profiling is the amount of memory used by functions and instructions. `memory_profiler` is a python module for monitoring memory consumption of a process as well as line-by-line analysis of memory consumption for Python programs. The purpose of this is to find memory leaks and optimize memory usage. Fortunately, it works similarly to `line_profiler`.


__References__
* [PyPi memory-profiler](https://pypi.org/project/memory-profiler/)

In [16]:
%load_ext memory_profiler

In [17]:
%memit?

Docstring:
Measure memory usage of a Python statement

Usage, in line mode:
  %memit [-r<R>t<T>i<I>] statement

Usage, in cell mode:
  %%memit [-r<R>t<T>i<I>] setup_code
  code...
  code...

This function can be used both as a line and cell magic:

- In line mode you can measure a single-line statement (though multiple
  ones can be chained with using semicolons).

- In cell mode, the statement in the first line is used as setup code
  (executed but not measured) and the body of the cell is measured.
  The cell body has access to any variables created in the setup code.

Options:
-r<R>: repeat the loop iteration <R> times and take the best result.
Default: 1

-t<T>: timeout after <T> seconds. Default: None

-i<I>: Get time information at an interval of I times per second.
    Defaults to 0.1 so that there is ten measurements per second.

-c: If present, add the memory usage of any children process to the report.

-o: If present, return a object containing memit run details

-q: If pres

In [18]:
%mprun?

Docstring:
Execute a statement under the line-by-line memory profiler from the
memory_profiler module.

Usage, in line mode:
  %mprun -f func1 -f func2 <statement>

Usage, in cell mode:
  %%mprun -f func1 -f func2 [statement]
  code...
  code...

In cell mode, the additional code lines are appended to the (possibly
empty) statement in the first line. Cell mode allows you to easily
profile multiline blocks without having to put them in a separate
function.

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through
the pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, o

In [ ]:
%%writefile profiling/memoryprofilerdemo.py

def some_sums(n):
    twos = [2] * (10**n)
    twosum = sum(twos)
    del twos
    # some long computation
    fives = [5] * (10**n)
    fivesum = sum(fives)
    del fives
    return twosum + fivesum


if __name__ == '__main__':
    some_sums(7)

In [ ]:
!python3 -m memory_profiler profiling/memoryprofilerdemo.py

In [ ]:
from profiling.memoryprofilerdemo import some_sums


%mprun -f some_sums some_sums(7)